In [1]:
import os

In [2]:
import pandas as pd

In [3]:
%pwd

'd:\\ML\\Data SC\\ML\\MLOPS try project\\salary-kaggle-project\\notebook'

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\ML\\Data SC\\ML\\MLOPS try project\\salary-kaggle-project'

In [6]:
data = pd.read_csv("artifacts/data_ingestion/salaries.csv")

In [7]:
data.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2025,SE,FT,Solutions Engineer,214000,USD,214000,US,100,US,M
1,2025,SE,FT,Solutions Engineer,136000,USD,136000,US,100,US,M
2,2025,MI,FT,Data Engineer,158800,USD,158800,AU,0,AU,M
3,2025,MI,FT,Data Engineer,139200,USD,139200,AU,0,AU,M
4,2025,EN,FT,Data Engineer,90000,USD,90000,US,0,US,M


In [8]:
data.isnull().sum()

work_year             0
experience_level      0
employment_type       0
job_title             0
salary                0
salary_currency       0
salary_in_usd         0
employee_residence    0
remote_ratio          0
company_location      0
company_size          0
dtype: int64

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np

In [11]:
data = data.drop(columns = ['salary', 'salary_currency'])

# Convert the 'work_year' column into a simpler format for the model
data['work_year'] = data['work_year'] - 2020


X = data.drop('salary_in_usd', axis=1)
y = data['salary_in_usd']

for col in ['job_title', 'employee_residence', 'company_location']:
    top_10 = X[col].value_counts().nlargest(10).index
    X[col] = np.where(X[col].isin(top_10), X[col], 'Other')

# Initial Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
numerical_features = ['work_year', 'remote_ratio']
ordinal_features = ['experience_level', 'company_size']
nominal_features = ['employment_type', 'job_title', 'employee_residence', 'company_location']

experience_order = ['EN', 'MI', 'SE', 'EX']
size_order = ['S', 'M', 'L']

        

In [12]:

preprocessor = ColumnTransformer(
transformers=[
        # Transformer for numerical features
        ('num', StandardScaler(), numerical_features),

        # Transformer for ordinal features
        ('ord', OrdinalEncoder(categories=[experience_order, size_order]), ordinal_features),

        # Transformer for nominal features
        ('cat', OneHotEncoder(handle_unknown='ignore', drop='first'), nominal_features)
    ],
    remainder='passthrough'
                                )       

preprocessor.fit(X_train)


,transformers,"[('num', ...), ('ord', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,copy,True
,with_mean,True
,with_std,True


In [13]:
X_train_transformed = preprocessor.transform(X_train)
# logger.info(f"X_train shape: {X_train_transformed.shape}")

X_test_transformed = preprocessor.transform(X_test)
# logger.info(f"X_test shape: {X_test_transformed.shape}")


feature_names = preprocessor.get_feature_names_out()

In [20]:
len(feature_names)

37

In [21]:
print("--- Debugging Information ---")
print(f"Shape of the data going into the DataFrame: {X_train_transformed.shape}")
print(f"Number of column names provided: {len(feature_names)}")

--- Debugging Information ---
Shape of the data going into the DataFrame: (106679, 37)
Number of column names provided: 37


In [22]:
X_train_df = pd.DataFrame(X_train_transformed.toarray(), columns=feature_names)
X_test_df = pd.DataFrame(X_test_transformed.toarray(), columns=feature_names)